In [1]:
import pandas as pd  
from random import random
 
pdata = pd.read_csv('oct_march.csv',names=['chid','views','subcriber','videocount','date']) 


In [2]:
pdata.drop(pdata.index[0])

,chid,views,subcriber,videocount,date
1,UC6ROKPXrnzfhNYST1w,3302429,2087,4,2016-10-01
2,UC6ROKPXrnzfhNYST1w,3304705,2087,4,2016-10-02
3,UC6ROKPXrnzfhNYST1w,3306526,2090,4,2016-10-03
4,UC6ROKPXrnzfhNYST1w,3309092,2090,4,2016-10-04
5,UC6ROKPXrnzfhNYST1w,3311343,2090,4,2016-10-05
6,UC6ROKPXrnzfhNYST1w,3313549,2091,4,2016-10-06
7,UC6ROKPXrnzfhNYST1w,3316519,2092,4,2016-10-07
8,UC6ROKPXrnzfhNYST1w,3319146,2091,4,2016-10-08
9,UC6ROKPXrnzfhNYST1w,3321635,2093,4,2016-10-09
10,UC6ROKPXrnzfhNYST1w,3324611,2094,4,2016-10-10


In [3]:
channel_groups=pdata.groupby('chid')
grps=[channel_groups.get_group(x) for x in channel_groups.groups]

In [ ]:

def _load_data(data, n_prev = 4):  
    """
    data should be pd.DataFrame()
    """

    docX, docY = [], []
    for i in range(len(data)-n_prev):
        docX.append(data.iloc[i:i+n_prev-1].as_matrix())
        docY.append(data.iloc[i+n_prev].as_matrix())
    alsX = np.array(docX)
    alsY = np.array(docY)

    return alsX, alsY

def train_test_split(df, test_size=0.25):  
    """
    This just splits data to training and testing parts
    """
    ntrn = round(len(df) * (1 - test_size))

    X_train, y_train = _load_data(df.iloc[0:ntrn])
    X_test, y_test = _load_data(df.iloc[ntrn:])

    return (X_train, y_train), (X_test, y_test)

In [ ]:
%matplotlib notebook
from keras.models import Sequential  
from keras.layers.core import Dense, Activation  
from keras.layers.recurrent import LSTM

import numpy as np
for i in range(len(grps)-1):
    p=grps[i]
    p=p.sort_values(by='date')
    del p['date']
    del p['chid']
    p['views']=p['views'].astype(float)
    views_range=p['views'].max()- p['views'].min()
    views_average=np.asarray(p['views'], dtype=np.float).mean()
    p['views']= (p['views'] - views_average )/ views_range
    p['subcriber']=p['subcriber'].astype(int)
    subcriber_range=p['subcriber'].max() - p['subcriber'].min()
    subcriber_average=np.asarray(p['subcriber'], dtype=np.float).mean()
    p['subcriber']= (p['subcriber'] - np.asarray(p['subcriber'], dtype=np.float).mean()) / (p['subcriber'].max() - p['subcriber'].min())
    p['videocount']=p['videocount'].astype(int)
    videocount_range=p['videocount'].max() - p['videocount'].min()
    videocount_average=np.asarray(p['videocount'], dtype=np.float).mean()
    p['videocount']= (p['videocount'] - np.asarray(p['videocount'], dtype=np.float).mean()) / (p['videocount'].max() - p['videocount'].min())
    (X_train, y_train), (X_test, y_test) = train_test_split(p)  # retrieve data
    X_train=X_train.reshape((-1,3,3))
    in_out_neurons = 3  

    hidden_neurons = 300


    model = Sequential()  

    model.add(LSTM(input_shape=(3,3),units=hidden_neurons, return_sequences=False))  

    model.add(Dense(units=10))  


    model.add(Activation("tanh"))


    model.add(Dense(units=3))

    model.add(Activation("linear"))

    model.compile(loss="mean_squared_error", optimizer="rmsprop")  
    model.fit(x=X_train, y=y_train,validation_split=0.05) 

Using TensorFlow backend.


Train on 125 samples, validate on 7 samples
Epoch 1/10
125/125 [==============================] - 0s - loss: nan - val_loss: nan
Epoch 2/10
125/125 [==============================] - 0s - loss: nan - val_loss: nan
Epoch 3/10
125/125 [==============================] - 0s - loss: nan - val_loss: nan
Epoch 4/10
125/125 [==============================] - 0s - loss: nan - val_loss: nan
Epoch 5/10
125/125 [==============================] - 0s - loss: nan - val_loss: nan
Epoch 6/10
125/125 [==============================] - 0s - loss: nan - val_loss: nan
Epoch 7/10
125/125 [==============================] - 0s - loss: nan - val_loss: nan
Epoch 8/10
125/125 [==============================] - 0s - loss: nan - val_loss: nan
Epoch 9/10
125/125 [==============================] - 0s - loss: nan - val_loss: nan
Epoch 10/10
125/125 [==============================] - 0s - loss: nan - val_loss: nan
Train on 125 samples, validate on 7 samples
Epoch 1/10
125/125 [==============================] - 0s - lo

125/125 [==============================] - 0s - loss: 0.0015 - val_loss: 3.4723e-04
Epoch 7/10
125/125 [==============================] - 0s - loss: 0.0014 - val_loss: 4.7392e-04
Epoch 8/10
125/125 [==============================] - 0s - loss: 0.0016 - val_loss: 4.2355e-04
Epoch 9/10
125/125 [==============================] - 0s - loss: 0.0033 - val_loss: 7.6729e-05
Epoch 10/10
125/125 [==============================] - 0s - loss: 0.0014 - val_loss: 1.9887e-04
Train on 125 samples, validate on 7 samples
Epoch 1/10
125/125 [==============================] - 1s - loss: 0.0263 - val_loss: 0.0309
Epoch 2/10
125/125 [==============================] - 0s - loss: 0.0049 - val_loss: 0.0082
Epoch 3/10
125/125 [==============================] - 0s - loss: 0.0013 - val_loss: 0.0014
Epoch 4/10
125/125 [==============================] - 0s - loss: 4.5830e-04 - val_loss: 4.3788e-04
Epoch 5/10
125/125 [==============================] - 0s - loss: 5.5519e-04 - val_loss: 1.3504e-04
Epoch 6/10
125/125 [

125/125 [==============================] - 1s - loss: nan - val_loss: nan
Epoch 2/10
125/125 [==============================] - 0s - loss: nan - val_loss: nan
Epoch 3/10
125/125 [==============================] - 0s - loss: nan - val_loss: nan
Epoch 4/10
125/125 [==============================] - 0s - loss: nan - val_loss: nan
Epoch 5/10
125/125 [==============================] - 0s - loss: nan - val_loss: nan
Epoch 6/10
125/125 [==============================] - 0s - loss: nan - val_loss: nan
Epoch 7/10
125/125 [==============================] - 0s - loss: nan - val_loss: nan
Epoch 8/10
125/125 [==============================] - 0s - loss: nan - val_loss: nan
Epoch 9/10
125/125 [==============================] - 0s - loss: nan - val_loss: nan
Epoch 10/10
125/125 [==============================] - 0s - loss: nan - val_loss: nan
Train on 125 samples, validate on 7 samples
Epoch 1/10
125/125 [==============================] - 1s - loss: 0.0265 - val_loss: 0.0170
Epoch 2/10
125/125 [=====

125/125 [==============================] - 0s - loss: 0.0108 - val_loss: 0.0092
Train on 125 samples, validate on 7 samples
Epoch 1/10
125/125 [==============================] - 1s - loss: nan - val_loss: nan
Epoch 2/10
125/125 [==============================] - 0s - loss: nan - val_loss: nan
Epoch 3/10
125/125 [==============================] - 0s - loss: nan - val_loss: nan
Epoch 4/10
125/125 [==============================] - 0s - loss: nan - val_loss: nan
Epoch 5/10
125/125 [==============================] - 0s - loss: nan - val_loss: nan
Epoch 6/10
125/125 [==============================] - 0s - loss: nan - val_loss: nan
Epoch 7/10
125/125 [==============================] - 0s - loss: nan - val_loss: nan
Epoch 8/10
125/125 [==============================] - 0s - loss: nan - val_loss: nan
Epoch 9/10
125/125 [==============================] - 0s - loss: nan - val_loss: nan
Epoch 10/10
125/125 [==============================] - 0s - loss: nan - val_loss: nan
Train on 125 samples, val

125/125 [==============================] - 0s - loss: 8.2787e-04 - val_loss: 4.3378e-04
Epoch 7/10
125/125 [==============================] - 0s - loss: 9.5126e-04 - val_loss: 9.9919e-04
Epoch 8/10
125/125 [==============================] - 0s - loss: 6.8768e-04 - val_loss: 0.0012
Epoch 9/10
125/125 [==============================] - 0s - loss: 0.0014 - val_loss: 0.0017
Epoch 10/10
125/125 [==============================] - 0s - loss: 7.4535e-04 - val_loss: 0.0018
Train on 125 samples, validate on 7 samples
Epoch 1/10
125/125 [==============================] - 2s - loss: 0.0553 - val_loss: 0.0117
Epoch 2/10
125/125 [==============================] - 0s - loss: 0.0159 - val_loss: 0.0042
Epoch 3/10
125/125 [==============================] - 0s - loss: 0.0130 - val_loss: 0.0024
Epoch 4/10
125/125 [==============================] - 0s - loss: 0.0134 - val_loss: 0.0013
Epoch 5/10
125/125 [==============================] - 0s - loss: 0.0125 - val_loss: 0.0016
Epoch 6/10
125/125 [============

125/125 [==============================] - 0s - loss: 0.0269 - val_loss: 0.0214
Epoch 3/10
125/125 [==============================] - 0s - loss: 0.0140 - val_loss: 0.0126
Epoch 4/10
125/125 [==============================] - 0s - loss: 0.0105 - val_loss: 0.0070
Epoch 5/10
125/125 [==============================] - 0s - loss: 0.0098 - val_loss: 0.0051
Epoch 6/10
125/125 [==============================] - 0s - loss: 0.0092 - val_loss: 0.0066
Epoch 7/10
125/125 [==============================] - 0s - loss: 0.0081 - val_loss: 0.0055
Epoch 8/10
125/125 [==============================] - 0s - loss: 0.0087 - val_loss: 0.0015
Epoch 9/10
125/125 [==============================] - 0s - loss: 0.0096 - val_loss: 0.0032
Epoch 10/10
125/125 [==============================] - 0s - loss: 0.0099 - val_loss: 0.0013
Train on 125 samples, validate on 7 samples
Epoch 1/10
125/125 [==============================] - 2s - loss: nan - val_loss: nan
Epoch 2/10
125/125 [==============================] - 0s - los

125/125 [==============================] - 0s - loss: 0.0019 - val_loss: 0.0011
Epoch 10/10
125/125 [==============================] - 0s - loss: 0.0013 - val_loss: 9.2286e-04
Train on 125 samples, validate on 7 samples
Epoch 1/10
125/125 [==============================] - 2s - loss: 0.0384 - val_loss: 0.0467
Epoch 2/10
125/125 [==============================] - 0s - loss: 0.0126 - val_loss: 0.0204
Epoch 3/10
125/125 [==============================] - 0s - loss: 0.0095 - val_loss: 0.0074
Epoch 4/10
125/125 [==============================] - 0s - loss: 0.0079 - val_loss: 0.0027
Epoch 5/10
125/125 [==============================] - 0s - loss: 0.0075 - val_loss: 7.7052e-04
Epoch 6/10
125/125 [==============================] - 0s - loss: 0.0077 - val_loss: 0.0017
Epoch 7/10
125/125 [==============================] - 0s - loss: 0.0069 - val_loss: 0.0020
Epoch 8/10
125/125 [==============================] - 0s - loss: 0.0069 - val_loss: 8.4964e-04
Epoch 9/10
125/125 [========================

125/125 [==============================] - 0s - loss: 0.0050 - val_loss: 0.0068
Epoch 4/10
125/125 [==============================] - 0s - loss: 0.0024 - val_loss: 0.0040
Epoch 5/10
125/125 [==============================] - 0s - loss: 0.0015 - val_loss: 0.0017
Epoch 6/10
125/125 [==============================] - 0s - loss: 0.0014 - val_loss: 0.0017
Epoch 7/10
125/125 [==============================] - 0s - loss: 0.0013 - val_loss: 0.0018
Epoch 8/10
125/125 [==============================] - 0s - loss: 0.0028 - val_loss: 0.0015
Epoch 9/10
125/125 [==============================] - 0s - loss: 0.0017 - val_loss: 0.0010
Epoch 10/10
125/125 [==============================] - 0s - loss: 0.0011 - val_loss: 9.0237e-04
Train on 125 samples, validate on 7 samples
Epoch 1/10
125/125 [==============================] - 2s - loss: 0.0810 - val_loss: 0.0180
Epoch 2/10
125/125 [==============================] - 0s - loss: 0.0146 - val_loss: 0.0106
Epoch 3/10
125/125 [==============================] 

In [ ]:
from datetime import timedelta, date

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

start_date = date(2017, 4, 1)
end_date = date(2017, 7, 1)
p=np.empty((100,3,3))
predicted=np.empty((100,3))
p[0]=grps[len(grps)-1].iloc[-3:].reshape((1,3,3))
for single_date in daterange(start_date, end_date):
    p[i][0]=p[i-1][1]
    p[i][1]=p[i-1][2]
    p[i][2]=model.predict(p[i-1].reshape((1,3,3)))
    predicted[i]=p[i][2]
    print (single_date,predicted[i])